# Customize your case: Parameter modifications
## *Customizing your case by changing model parameters*

**This is an optional tutorial** that provides some additional guidance for creating an experimental clone for a NEON case and testing parameter sensitivities in the model.  Once you settle on a new parameter set you want to use, you should also spinning up the model.  These examples are in the [customizeCase_spinupExample notebook](./customizeCase_spinupExample.ipynb)

We're going to move kind of quickly through some material that's already covered in the tutorial on [cloning a NEON case](./Clone_Case.ipynb).  The cloned case that you created in the Clone_Case tutorial will be used as the control simulation for this model experiment. **If you have tried cloning a case you need to do that now.**

Additionally, this tutorial assumes that you've modified the CLM parameter file.  We provided an example notebook for [modifying parameters](./modifyParameterFile.ipynb).  **This should be done before running the rest of this notebook.**
<br>

---
## In this tutorial

1. Create an experimental clone for a NEON site that you've already run.
2. Point to a modified parameter file in user_nl_clm.
3. Submit the case. 

---
### First let's start with a bit of background.

Parameter files are read into CLM.  We provide a default parameter file for your cases, but this can be modified by pointing to a new parameter file in user_nl_clm.  

Since changing parameters in the model will change answers, we also need to spin up the model to generate new initial conditions files.

<div class="alert alert-block alert-warning">

<b>NOTICE:</b> If you're running this notebook through the NCAR JupyterHub login, you need to be on a Cheyenne login node (NOT Casper).  

</div>

### Set up your environment
It is important in order to have all the tools and packages you need to run simulations. 

<div class="alert alert-block alert-info">
   
<b>TIP:</b> <i>If you're running on Cheyenne</i>, you may need to uncomment the the following two lines of code. This will set up your conda environment correctly.

<b>This is not required if your running CESM-Lab in the cloud.</b>

</div>

In [ ]:
#module purge
#module load conda ncarenv

# 1. Create an experimental clone.
This tutorial assumes that you've already run a control case for the NEON site you're interested in. 

## 1.1 Check that you've run a control case.

In [ ]:
export neon_site='HARV'            # the site you want to run
ls ~/scratch/NEON_cases/$neon_site.transient

If the code block above says *"No such file or directory"* you need to go back and create a control case for your neon_site using [clone_case](Clone_Case.ipynb).

If you have run a control case, which includes flies like: 
- case.build, case.setup, and case.submit; 
- env_build.xml, env_batch.xml, and env_run.xml; and 
- user_nl_clm, user_nl_datm, and user_nl_datm_streams  

Then you can move on to the next step

## 1.2 Navigate to your source code

In [ ]:
cd ~/CTSM/tools/site_and_regional

## 1.3 Create an experimental clone

In our experimental case we'll changing the foliar C:N ratio for a temperate deciduous tree. 

<div class="alert alert-block alert-info">

<b>HINT:</b> It's helpful to have descriptive experiment names. 

</div>


In [ ]:
export experiment='foliarCN-30'    # this is name of your experimenal case 

Since we're creating an experimental case that will change answers in the model we should run a full AD and postAD spinup, but for now we just want to run a quick sensitivity experiment with a new parameter file.


Then run_neon with the following flags:
- `--experiment $experiment` 
- `--setup-only` 

To make things faster, we'll also use the experimental control case as our 'base_case'.

In [ ]:
./run_neon.py --neon-sites $neon_site  \
   --output-root ~/scratch/NEON_cases \
   --base-case ~/scratch/NEON_cases/$neon_site.transient \
   --experiment $experiment \
   --overwrite \
   --setup-only  
   
echo '----Your experimental clone has been created----'

# 2 Customizing your case: Namelist changes
The configuration of CLM can be customized via namelist modifications. 

These changes made using namelist files like `user_nl_clm`

## 2.1 Move to the case directory for your experimental clone

In [ ]:
cd ~/scratch/NEON_cases/$neon_site.$experiment.transient

## 2.2 Getting familiar with namelist files
`user_nl_<model>` files are created in the case directory after setting up your case with `./case.setup`
  - Note: This was already done for us since we cloned a case with run_neon
    
#### Take a look!
  - What other user `user_nl_<model>` files do you already have in this case directory?
  - What kind of information is already included in a `user_nl_clm` file?

We can explore these files on the comand line using an editor, by opening them directly in jupyter hub, or by simply printing the contents of the files to the screen here.

In [ ]:
ls user_nl_*

In [ ]:
cat user_nl_clm | tail -18

The compset that you created your case with sets up initial, or default namelist options. These can be found in `CaseDocs/lnd_in`

<style> 
table td, table th, table tr {text-align:left !important;}
</style>
<div class="alert alert-block alert-warning">
<b>Important:</b>   Don’t modify the lnd_in namelist file directly. Instead, make changes in user_nl_clm.
</div>

## 2.3 Looking deeper at namelist options 
All CLM namelist options are defined in the `lnd_in` file

We can explore this file on the comand line using an editor, by opening it directly in jupyter hub, or by simply printing the contents of the files to the screen here.  
- **NOTE:** This file is huge, so we'll just look at the first 65 lines

In [ ]:
cat CaseDocs/lnd_in | head -40

There's a lot to dig into here! So we'll just stick with a few highlights.

#### Take a look
**See what printed to in the cell above and answer the following questions**
- `finidat`: What initial conditions file are you using? 
- `fsurdat`: What surface dataset are you using?
- `paramfile`: What parameter file are you using?

Does this make sense?  Answers are in the hidden cell below.


- `finidat = '.../HARV.2022-11-11.clm2.r.0318-01-01-00000.nc'`  This is for a control case, not one with modified parameters!  That's OK for now, but it's good to be aware of where your initial conditions are coming from.
- `fsurdat = ".../surfdata_1x1_NEON_HARV_hist_78pfts_CMIP6_simyr2000_c230111.nc"` This the surface dataset that's been modified for our NEON site.
- `paramfile = '.../ctsm51_params.c211112.nc'` is the default parameter file for CTSM5.1  

***

- Additional information about customizing CTSM's configurations, including namelist modifications, are available in the [CTSM users guide](https://escomp.github.io/ctsm-docs/versions/master/html/users_guide/setting-up-and-running-a-case/customizing-the-clm-configuration.html?highlight=namelist)
Namelist changes can also be used to modify variables on history file output. Simple modification of history file output DOES NOT require spinning up the model.
- A list of all the [CTSM history fields are available here](https://escomp.github.io/ctsm-docs/versions/master/html/users_guide/setting-up-and-running-a-case/master_list_nofates.html)
- A list of all the [CTSM-FATES history fields are available here](https://escomp.github.io/ctsm-docs/versions/master/html/users_guide/setting-up-and-running-a-case/master_list_fates.html)

## 2.4 Point to your modified parameter file

For this experiment we're changing the foliar C:N ratio for temperate deciduous forests.  Do do this we have to point to our modified parameter file. We can change user_nl_clm using an editor, by opening it directly in jupyter hub, or with the following command.

In [ ]:
# This is the path to the modified parameter file you created

# If you're running on cheyenne, uncomment the line below
#echo "paramfile = '/glade/scratch/$USER/NEON_cases/modified_inputs/ctsm51_params.c211112_tdf_leafcn30.nc'" >> user_nl_clm

# If you're running with CESM-Lab in the cloud, uncomment the line below
echo "paramfile = '/scratch/$USER/NEON_cases/modified_inputs/ctsm51_params.c211112_tdf_leafcn30.nc'" >> user_nl_clm

Let make sure this worked.  We can:  
- preview namelists;
- check the  `lnd_in` file.

In [ ]:
./preview_namelists

In [ ]:
cat CaseDocs/lnd_in | grep paramfile

We've changes ethe parameter file that's being used for our simulation.

# 3 Sumbit the run
We already build the model, and when making namelist or .xml changes we don't have to rebuild our case, so now we're ready to submit our simulation.

In [ ]:
./case.submit

Now our case is submitted!  Check that your experimental case is running 

In [ ]:
qstat -u $USER

In [ ]:
tail  ~/scratch/NEON_cases/$neon_site.$experiment.transient/CaseStatus

<div class="alert alert-success">
<strong>Congratulations!</strong> 
    
You've created an experimental clone of CLM at the NEON tower you selected!
    
Now we need to see how the simulations are different.
</div>

**Assuming you see differences between your experimental and control cases, you may want to consider spinning up the experimental case [Spinup Example notebook](./customizeCase_spinupExample.ipynb).**

If you're developing this tutorial:
Before saving and pushing this code to github go to `Kernel` and `Restart kernel and clear all outputs...`